In [91]:
drive=True
if drive:
    from google.colab import drive
    drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# Font Classification Project

In [92]:
import tensorflow as tf
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))
# tf.config.run_functions_eagerly(True)

Num GPUs Available:  0


Working with tensorflow and local GPU (GTX 1070)

## Functions for data generation

In [93]:
import h5py
import matplotlib.pyplot as plt
import cv2
import numpy as np
import sklearn
import sklearn.model_selection

In [94]:
input_height=224
input_width=224

In [95]:
def fonts_to_output(fonts, label_encoder):
    label = label_encoder.transform(fonts)
    return tf.keras.utils.to_categorical(label, 7)

In [96]:
def crop_wrap_letter(img, bounding_box):
    src=bounding_box
    input_pts = np.float32(src)
    # y comes first in the matrix
    output_pts = np.float32([[0, 0],
                            [input_height - 1, 0],
                            [input_height - 1, input_width - 1],
                            [0, input_width - 1]])
    # Compute the perspective transform M
    M = cv2.getPerspectiveTransform(input_pts,output_pts)
    out = cv2.warpPerspective(img,
                          M,
                          (input_width, input_height),
                          flags=cv2.INTER_LINEAR)
    return out

In [97]:
def crop_wrap_image(img, txts, wordBB, fonts, label_encoder=None):
    mini_batch = []
    mini_batch_fonts = []
    i=0
    for j in range(len(txts)):
        txt = txts[j]
        font = fonts[i]
        bounding_box = np.dstack(wordBB[:,:,j])[0]  # pairs of x,y
        crop = tf.keras.applications.resnet50.preprocess_input(crop_wrap_letter(img, bounding_box))  # Identify as 1 batch of image
        mini_batch_fonts.append(font)
        mini_batch.append(crop)
        crop = crop.reshape((1,input_height,input_width,3))  # Identify as 1 batch of image
        # if not label_encoder:
        #     yield crop
        # else:
        #     yield crop, fonts_to_output([font], label_encoder)
        i+=len(txt)
    mini_batch = np.asarray(mini_batch)
    if not label_encoder:
        yield mini_batch
    else:
        yield mini_batch, fonts_to_output(mini_batch_fonts, label_encoder)

In [98]:
def crop_wrap_data(db, im_names, label_encoder=None):
    for im in im_names:
        img = db['data'][im][:]
        fonts = db['data'][im].attrs['font']
        txt = db['data'][im].attrs['txt']
        charBB = db['data'][im].attrs['charBB']
        wordBB = db['data'][im].attrs['wordBB']
        
        yield from crop_wrap_image(img, txt, wordBB, fonts, label_encoder)

In [99]:
def data_generator(*args, **kwargs):
    while True:
        yield from crop_wrap_data(*args, **kwargs)

In [100]:
def font_label_data(db, im_names, label_encoder=None):
    for im in im_names:
        font = db['data'][im].attrs['font']
        txt = db['data'][im].attrs['txt']
        i=0
        for j in range(len(txt)):
            f = font[i]
            if not label_encoder:
                yield f
            else:
                yield fonts_to_output([f], label_encoder)
            i+=len(txt[j])

In [101]:
def load_db():
    path = 'SynthText.h5' if not drive else '/content/drive/MyDrive/Fonter/SynthText.h5'
    db = h5py.File(path, 'r')
    im_names = list(db['data'].keys())
    return db, im_names

In [102]:
def data_size(db, im_names):
    return len(im_names)

## Prepare the data

In [103]:
db, im_names = load_db()

In [104]:
label_encoder = sklearn.preprocessing.LabelEncoder()
label_encoder.fit(list(font_label_data(db, im_names)))
label_encoder.classes_, len(label_encoder.classes_)

(array([b'Alex Brush', b'Michroma', b'Open Sans', b'Raleway', b'Roboto',
        b'Russo One', b'Ubuntu Mono'], dtype='|S14'), 7)

In [105]:
#partial_data, _ = sklearn.model_selection.train_test_split(im_names, test_size=0.99, random_state = 42)  # testing purposes

In [106]:
X_train, X_test = sklearn.model_selection.train_test_split(im_names, test_size=0.2, random_state = 42)

In [107]:
X_train, X_val = sklearn.model_selection.train_test_split(X_train, test_size=0.1, random_state = 42)

In [108]:
train_data_and_labels = data_generator(db, X_train, label_encoder=label_encoder)
val_data_and_labels = data_generator(db, X_val, label_encoder=label_encoder)
test_data = data_generator(db, X_test)
test_labels = list(font_label_data(db, X_test))

In [109]:
# size = data_size(db, X_train)
# for i, (img, label) in enumerate(train_data_and_labels):
#     if i>=size*3:
#         break
#     if i%size!=0:
#         continue
#     plt.figure()
#     plt.imshow(img.reshape((input_height,input_width,3)))
#     plt.title(label[0])

## Define a model

In [110]:
resnet = tf.keras.applications.resnet50.ResNet50(include_top=False, weights='imagenet', input_shape=(input_height,input_width,3))

In [111]:
# Freeze the layers which you don't want to train. Here I am freezing the all layers.
resnet.trainable = False
resnet.training = False

In [112]:
# inputs = tf.keras.Input(shape=(input_height, input_width, 3))
# x = resnet(inputs, training=False)
# # x = tf.keras.layers.GlobalAveragePooling2D()(x)
# # x = tf.keras.layers.Dropout(0.2)(x)
# outputs = tf.keras.layers.Dense(len(label_encoder.classes_), activation='softmax')(x)
# fonter = tf.keras.Model(inputs, outputs)

# # # ###could be defined as Sequential I think..

In [113]:
fonter = tf.keras.models.Sequential([
    tf.keras.layers.Rescaling(1./255, input_shape=(input_height,input_width,3)),
    resnet,
    tf.keras.layers.GlobalAveragePooling2D(),
    tf.keras.layers.Dense(512, activation='relu'),
    #tf.keras.layers.Dropout(0.5),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dense(512, activation='relu'),
    #tf.keras.layers.Dropout(0.5),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dense(len(label_encoder.classes_), activation="softmax"),
])

In [114]:
base_learning_rate = 0.001
opt = tf.keras.optimizers.Nadam(learning_rate=base_learning_rate)
fonter.compile(optimizer=opt,
              loss = tf.keras.losses.CategoricalCrossentropy(from_logits=False),
              metrics=['accuracy'])

In [115]:
fonter.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 rescaling_3 (Rescaling)     (None, 224, 224, 3)       0         
                                                                 
 resnet50 (Functional)       (None, 7, 7, 2048)        23587712  
                                                                 
 global_average_pooling2d_3   (None, 2048)             0         
 (GlobalAveragePooling2D)                                        
                                                                 
 dense_9 (Dense)             (None, 512)               1049088   
                                                                 
 batch_normalization_6 (Batc  (None, 512)              2048      
 hNormalization)                                                 
                                                                 
 dense_10 (Dense)            (None, 512)              

## Train Model

In [116]:
x= next(crop_wrap_data(db, X_val, label_encoder=label_encoder))

In [117]:
x[0].shape, x[1].shape

((8, 224, 224, 3), (8, 7))

In [118]:
initial_epochs = 10
loss0, accuracy0 = fonter.evaluate(crop_wrap_data(db, X_val, label_encoder=label_encoder))

78/78 [==============================] - 96s 1s/step - loss: 2.0017 - accuracy: 0.1544


In [119]:
# print("initial loss: {:.2f}".format(loss0))
# print("initial accuracy: {:.2f}".format(accuracy0))

In [120]:
# d=list(crop_wrap_data(db, X_train, label_encoder))

In [121]:
# imgs=np.array([x[0][0] for x in d])
# labels=np.array([x[1][0] for x in d])

In [122]:
# imgs.shape

In [123]:
# labels.shape

In [124]:
# history = fonter.fit(imgs, labels, batch_size=32,
#                     epochs=100,
#                     #validation_data=val_data_and_labels,
#                     #max_queue_size=1,
#                     #steps_per_epoch=data_size(db, X_train),
#                     #validation_steps=data_size(db, X_val)
#                     )

In [125]:
history = fonter.fit(train_data_and_labels,
                    epochs=initial_epochs,
                    validation_data=val_data_and_labels,
                    max_queue_size=1,
                    steps_per_epoch=data_size(db, X_train),
                    validation_steps=data_size(db, X_val)
                    )

Epoch 1/10
700/700 [==============================] - 959s 1s/step - loss: 1.9635 - accuracy: 0.2484 - val_loss: 8.0778 - val_accuracy: 0.1939
Epoch 2/10
700/700 [==============================] - 964s 1s/step - loss: 1.8342 - accuracy: 0.2983 - val_loss: 3.3782 - val_accuracy: 0.2208
Epoch 3/10
700/700 [==============================] - 946s 1s/step - loss: 1.7701 - accuracy: 0.3253 - val_loss: 9.2897 - val_accuracy: 0.1544
Epoch 4/10
700/700 [==============================] - 949s 1s/step - loss: 1.7309 - accuracy: 0.3418 - val_loss: 10.3838 - val_accuracy: 0.1526
Epoch 5/10
700/700 [==============================] - 970s 1s/step - loss: 1.6930 - accuracy: 0.3585 - val_loss: 13.3089 - val_accuracy: 0.1616
Epoch 6/10
700/700 [==============================] - 982s 1s/step - loss: 1.6661 - accuracy: 0.3707 - val_loss: 4.9322 - val_accuracy: 0.1939
Epoch 7/10
700/700 [==============================] - 971s 1s/step - loss: 1.6433 - accuracy: 0.3754 - val_loss: 6.5813 - val_accuracy: 0.20

In [2]:
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']

loss = history.history['loss']
val_loss = history.history['val_loss']

plt.figure(figsize=(8, 8))
plt.subplot(2, 1, 1)
plt.plot(acc, label='Training Accuracy')
plt.plot(val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.ylabel('Accuracy')
plt.title('Training and Validation Accuracy')

plt.subplot(2, 1, 2)
plt.plot(loss, label='Training Loss')
plt.plot(val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.ylabel('Cross Entropy')
plt.title('Training and Validation Loss')
plt.xlabel('epoch')
plt.show()

NameError: ignored

In [128]:
loss, accuracy = fonter.evaluate(crop_wrap_data(db, X_test, label_encoder=label_encoder))

195/195 [==============================] - 236s 1s/step - loss: 10.1776 - accuracy: 0.1594


In [ ]:
fonter.evaluate(imgs, labels, batch_size=1)

In [ ]:
predictions = fonter.predict(imgs)

In [ ]:
np.argmax(predictions, axis=1)

In [ ]:
np.argmax(labels, axis=1)

In [ ]:
for i, (img, label) in enumerate(val_data_and_labels):
    if i==data_size(db, X_val)*2:
        break
    plt.figure()
    plt.imshow(img.reshape((input_height,input_width,3)))
    plt.title(label[0])

In [ ]:
X_train

In [ ]:
im_names.index(X_val[0])

In [ ]:
im_names.index(X_train[-1])

In [ ]:
im_names.index(X_test[0])